# Quadrotor (aka "the drone")

## Example of how to set everything up (do this only once)

Import modules and configure the notebook.

In [ ]:
import os

# This module is part of the python standard library
import time

# These modules are part of other existing libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy import linalg

# This is my own script (it is an interface to the pybullet simulator)
import ae353_drone

# I often go back and forth between making changes to my scripts and to
# the notebook in which they are used. One "gotcha" is that notebooks only
# import modules or scripts ONCE. Subsequent imports don't do anything, and
# in particular won't reflect any changes I've made to my scripts. To make
# sure that I'm working with the latest version of my code, I use this bit
# of magic, which forces the notebook to "reload" my script:
import importlib
importlib.reload(ae353_drone)

Create simulator.

In [ ]:
simulator = ae353_drone.Simulator(display=True, pos_noise = 0.1, rpy_noise = 0.1)

## Example of how to add a drone and run a simulation

In [ ]:
x_des_array = []

Define a controller for the drone.

In [ ]:
class RobotController:
    def __init__(self, limiter=None):
        self.dt = 0.01
        self.limiter = limiter
        self.A = np.array([[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, -0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, 0.0, 1.0],
 [0.0, 0.0, 0.0, 0.0, 9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, -9.81, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, -0.0, -0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, -0.0, -0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]])
        self.B = np.array([[0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 0.0, -0.0],
 [0.0, 0.0, 0.0, 2.0],
 [434.7826086956522, 0.0, 0.0, 0.0],
 [0.0, 434.7826086956522, 0.0, 0.0],
 [0.0, 0.0, 250.0, 0.0]])
        self.C = np.array([[1,0,0,0,0,0,0,0,0,0,0,0],
              [0,1,0,0,0,0,0,0,0,0,0,0],
              [0,0,1,0,0,0,0,0,0,0,0,0],
              [0,0,0,1,0,0,0,0,0,0,0,0],
              [0,0,0,0,1,0,0,0,0,0,0,0],
              [0,0,0,0,0,1,0,0,0,0,0,0]])
        self.K = np.array([[-7.470103824860964e-16,
  -0.597614304667193,
  -2.534736450492715e-16,
  2.131279246713305,
  -2.2795088647501232e-15,
  1.2174213473461131e-14,
  -5.725766455299804e-16,
  -0.6115194547658369,
  2.1235858790732371e-16,
  0.3522635360360125,
  1.882304521121369e-17,
  9.777282575744851e-17],
 [0.26726124191242473,
  -2.9636468334901435e-17,
  3.3383667491262037e-16,
  -4.687289900146584e-16,
  1.7324268494946549,
  4.0340005770852824e-17,
  0.3892794424612655,
  4.4031235194257394e-16,
  3.996567687509206e-18,
  1.882304521121369e-17,
  0.3496496500690224,
  -3.7084882309639604e-17],
 [-2.4083942781012557e-16,
  -9.452569554399674e-15,
  -1.6235935707886185e-16,
  9.560492828896679e-15,
  7.267272416605851e-16,
  0.5855400437691188,
  1.0873236001454141e-16,
  -3.634259171884253e-15,
  -9.553253349797393e-17,
  5.621937481053289e-17,
  -2.1323807328042768e-17,
  0.34492033085318],
 [-8.423703219100551e-17,
  -1.1812055222399395e-15,
  1.581138830084189,
  1.754554067453382e-15,
  -7.341405092272812e-16,
  -7.782195460359621e-16,
  -2.3199499377039327e-16,
  -1.0143593903012036e-15,
  1.8923897141139292,
  3.418973265307912e-17,
  6.434473976889822e-19,
  -2.6749109379432702e-17]])
        self.L = np.array(
[[11.3058803110068, 0.0, 0.0, 0.0, 0.4396776833728971, 0.0],
 [0.0, 11.3058803110068, 0.0, -0.4396776833728971, 0.0, 0.0],
 [0.0, 0.0, 10.954451150103324, 0.0, 0.0, 0.0],
 [0.0, -0.4396776833728971, 0.0, 10.945603950560619, 0.0, 0.0],
 [0.4396776833728971, 0.0, 0.0, 0.0, 10.945603950560619, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 10.954451150103324],
 [14.00812303603352, 0.0, 0.0, 0.0, 9.717323102417483, 0.0],
 [0.0, 14.00812303603352, 0.0, -9.717323102417483, 0.0, 0.0],
 [0.0, 0.0, 10.0, 0.0, 0.0, 0.0],
 [0.0, -0.06615794931719185, 0.0, 9.999781153892402, 0.0, 0.0],
 [0.06615794931719185, 0.0, 0.0, 0.0, 9.999781153892402, 0.0],
 [0.0, 0.0, 0.0, 0.0, 0.0, 10.0]])
        self.f_z_e = 9.81/2

    def get_color(self):
        return [1., 0., 0.]

    def reset(self, pos):
        self.xhat = np.array([[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.]])


    def run(self, pos, rpy, pos_ring, is_last_ring, pos_others):
        offset = -0.3
        y = np.array([[pos[0]], [pos[1]], [pos[2]], [rpy[0]], [rpy[1]], [rpy[2]]])
        dist = linalg.norm([[pos_ring[0] + offset - pos[0]], [pos_ring[1] - pos[1]], [pos_ring[2] - pos[2]]])
        displace_vec = ([[pos_ring[0] + offset - pos[0]], [pos_ring[1] - pos[1]], [pos_ring[2] - pos[2]]])/dist
        if dist < 1.:
            v_vect = np.array([[3.], [0.], [0.]])
        else:
            # v_vect = displace_vec * (dist)**2 * 0.1
            v_vect =displace_vec * 3.5

        # if dist < 0.5:
        #     offset = 2.
        # elif dist < 1.5:
        #     offset = -0.3
        # elif dist < 3:
        #     offset = -0.6    
        # x_des = np.array([[pos_ring[0] + offset],[pos_ring[1]],[pos_ring[2]],[0.],[0.],[0.],[v_vect[0,0]],[v_vect[1,0]],[v_vect[2,0]],[0.],[0.],[0.]])
        x_des = np.array([[pos_ring[0] + offset],[pos_ring[1]],[pos_ring[2]],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.],[0.]])
        # if is_last_ring:
        #     x_des[2,0] += 0.4

        
        # if dist < 0.5:
        #     x_des[7,0] = -x_des[7,0] # Setting vy_des to zero to reduce final momentum
        # if (pos_ring[0] - 0.3 - pos[0] < 0.4) and (pos_ring[1] - pos[1] < 0.4):
        #     x_des[6,0] = 1.5
        max_error = 2.3

        for i in range(12):
            if np.abs(x_des[i,0] - self.xhat[i,0]) > max_error:
                x_des[i,0] = self.xhat[i,0] + max_error * ((x_des[i,0] - self.xhat[i,0]) / linalg.norm(x_des[i,0] - self.xhat[i,0]))
        
        x_des_array.append(x_des)

        u = -self.K @ (self.xhat - x_des)
        tau_x = u[0,0]
        tau_y = u[1,0]
        tau_z = u[2,0]
        if y[2,0] <= 0.5:
            u[3,0] = 2.5
        if is_last_ring:
            if abs(pos_ring[0] - pos[0]) < 0.6 and abs(pos_ring[1] - pos[1]) < 0.6:
                u[3,0] = -2.5

        f_z = u[3,0] + self.f_z_e
        if self.limiter is not None:
            tau_x, tau_y, tau_z, f_z = self.limiter(tau_x, tau_y, tau_z, f_z)
        u[3,0] = f_z - self.f_z_e
        self.xhat += self.dt * (self.A @ self.xhat + self.B @ u - self.L @ (self.C @ self.xhat - y))

        return tau_x, tau_y, tau_z, f_z

Add the drone to the simulator. There are three arguments:

* `RobotController` is the class definition
* `'my_netid'` is the name of the drone
* `'my_image.png'` is the name of an image to use as a label (can be `None`, can be a file path)

You can call this function more than once, so long as you call it with a different drone name each time. An exception will be raised if you try to add two drones of the same name to the simulator.

In [ ]:
simulator.add_drone(RobotController, 'my_netid', 'my_image.png')

Reset the drone (i.e., place it at a random location in the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until the drone reaches the finish ring.

In [ ]:
simulator.camera_droneview('my_netid')

In [ ]:
num_iters = 100
finished = 0
failed = 0
finish_times = np.zeros(num_iters) - np.ones(num_iters)
for i in range(num_iters):
    simulator.clear_drones()
    simulator.add_drone(RobotController, 'my_netid', 'my_image.png')
    # simulator.load_drones()    
    simulator.camera_droneview('my_netid')
    simulator.move_rings()
    simulator.reset()
    simulator.run(max_time=70.0)
    drone_name = 'my_netid'
    drone = simulator.get_drone_by_name(drone_name)
    if drone['finish_time'] is None:
        if drone['running']:
            failed += 1
        else:
            failed += 1 
    else:
        finished += 1
        finish_times[i] = drone['finish_time']
    if i%10 == 0:
        print(f"{i} / 100")

In [ ]:
finished

In [ ]:
import matplotlib.pyplot as plt
actual_finish_times = []
for i in range(len(finish_times)):
    if finish_times[i] >= 0:
         actual_finish_times.append(finish_times[i])
actual_finish_times = np.array(actual_finish_times)
print(np.mean(actual_finish_times))
print(np.median(actual_finish_times))
plt.hist(actual_finish_times)

In [ ]:
simulator.run(max_time=30.0)

## Example of how to get and plot results

Assume that we already did two things:

* Added a drone with the name `my_netid` to the simulator
* Ran the simulation

Get drone by name.

In [ ]:
drone_name = 'my_netid'
drone = simulator.get_drone_by_name(drone_name)

if drone is None:
    drone_names = '\n'.join([d['name'] for d in simulator.drones])
    msg = f'The simulator has no drone with name "{drone_name}".'
    if len(drone_names) == 0:
        msg += f' The simulator has no drones at all, in fact.'
    else:
        msg += f' The simulator has these drones:'
        msg += f'\n==========\n{drone_names}\n==========\n'
    print(msg)

Extract data.

In [ ]:
data = drone['data'].copy()

Convert all lists in data to numpy arrays.

In [ ]:
for key in data.keys():
    data[key] = np.array(data[key]).T

Did the drone finish? If so, what was the time at which it finished? If not, was it still running (or was there likely an error)?

In [ ]:
if drone['finish_time'] is None:
    if drone['running']:
        print(f'The drone "{drone["name"]}" did not finish.')
    else:
        print(f'The drone "{drone["name"]}" did not finish and was no longer running.')
else:
    print(f'The drone "{drone["name"]}" finished with time: {drone["finish_time"]}')

In [ ]:
x_des_array = np.array(x_des_array)
x_des_array[:,0,0]

Plot results.

In [ ]:
# Create a figure with subplots that all share the same x-axis
fig, (ax_pos, ax_d, ax_d2, ax_v, ax_rpy, ax_taus, ax_fz) = plt.subplots(7, 1, figsize=(16, 30), sharex=True)

# Position
ax_pos.plot(data['t'], data['pos'][0, :], label='x (m)', linewidth=4)
ax_pos.plot(data['t'], data['pos'][1, :], label='y (m)', linewidth=4)
ax_pos.plot(data['t'], data['pos'][2, :], label='z (m)', linewidth=4)
ax_pos.plot(data['t'], data['xhat'][0, :], '--', label='x_est (m)', linewidth=4)
ax_pos.plot(data['t'], data['xhat'][1, :], '--', label='y_est (m)', linewidth=4)
ax_pos.plot(data['t'], data['xhat'][2, :], '--', label='z_est (m)', linewidth=4)
ax_pos.grid()
ax_pos.legend(fontsize=16)
ax_pos.tick_params(labelsize=14)


# Desired
ax_d.plot(data['t'], data['pos'][0, :], label='x (m)', linewidth=4)
ax_d.plot(data['t'], data['pos'][1, :], label='y (m)', linewidth=4)
ax_d.plot(data['t'], data['pos'][2, :], label='z (m)', linewidth=4)
ax_d.plot(data['t'], x_des_array[:,0,0], '--', label='x_des (m)', linewidth=4)
ax_d.plot(data['t'], x_des_array[:,1,0], '--', label='y_des (m)', linewidth=4)
ax_d.plot(data['t'], x_des_array[:,2,0], '--', label='z_des (m)', linewidth=4)
ax_d.grid()
ax_d.legend(fontsize=16)
ax_d.tick_params(labelsize=14)

# Desired vs Estimate
ax_d2.plot(data['t'], data['xhat'][0, :], '*', label='x_est (m)', linewidth=4)
ax_d2.plot(data['t'], data['xhat'][1, :], '*', label='y_est (m)', linewidth=4)
ax_d2.plot(data['t'], data['xhat'][2, :], '*', label='z_est (m)', linewidth=4)
ax_d2.plot(data['t'], x_des_array[:,0,0], '--', label='x_des (m)', linewidth=4)
ax_d2.plot(data['t'], x_des_array[:,1,0], '--', label='y_des (m)', linewidth=4)
ax_d2.plot(data['t'], x_des_array[:,2,0], '--', label='z_des (m)', linewidth=4)
ax_d2.grid()
ax_d2.legend(fontsize=16)
ax_d2.tick_params(labelsize=14)
#Velocity

ax_v.plot(data['t'], data['linvel'][0, :], label='vx (m)', linewidth=4)
ax_v.plot(data['t'], data['linvel'][1, :], label='vy (m)', linewidth=4)
ax_v.plot(data['t'], data['linvel'][2, :], label='vz (m)', linewidth=4)
ax_v.plot(data['t'], data['xhat'][6, :], '--', label='vx_est (m)', linewidth=4)
ax_v.plot(data['t'], data['xhat'][7, :], '--', label='vy_est (m)', linewidth=4)
ax_v.plot(data['t'], data['xhat'][8, :], '--', label='vz_est (m)', linewidth=4)
ax_v.set_ylim(-4,4)
ax_v.grid()
ax_v.legend(fontsize=16)
ax_v.tick_params(labelsize=14)


# Roll, pitch, and yaw angles
ax_rpy.plot(data['t'], data['rpy'][0, :], label='roll (rad)', linewidth=4)
ax_rpy.plot(data['t'], data['rpy'][1, :], label='pitch (rad)', linewidth=4)
ax_rpy.plot(data['t'], data['rpy'][2, :], label='yaw (rad)', linewidth=4)
ax_rpy.grid()
ax_rpy.legend(fontsize=16)
ax_rpy.tick_params(labelsize=14)

# Actuator commands
ax_taus.plot(data['t'], data['tau_x'], label='tau_x (N-m)', linewidth=4)
ax_taus.plot(data['t'], data['tau_y'], label='tau_y (N-m)', linewidth=4)
ax_taus.plot(data['t'], data['tau_z'], label='tau_z (N-m)', linewidth=4)
ax_taus.grid()
ax_taus.legend(fontsize=16)
ax_taus.tick_params(labelsize=14)

# Force command 
ax_fz.plot(data['t'], data['f_z'], label='f_z (N)', linewidth=4)
ax_fz.grid()
ax_fz.legend(fontsize=16)
ax_fz.tick_params(labelsize=14)

# Set shared x-axis properties
ax_fz.set_xlabel('time (s)', fontsize=20)
ax_fz.set_xlim([data['t'][0], data['t'][-1]])

# Make the arrangement of subplots look nice
fig.tight_layout()

## Example of how to load drones from the "students" directory

Load drones from `students` directory.

In [ ]:
simulator.load_drones()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to show results, change views, keep running

Show results (so far).

In [ ]:
simulator.show_results()

View from behind a particular drone.

In [ ]:
simulator.camera_droneview('template')

Continue running simulation from same point for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

View from finish ring instead of from start ring.

In [ ]:
simulator.camera_finishview()

Continue running simulation from same point for `max_time` seconds or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

Show results (so far).

In [ ]:
simulator.show_results()

## Example of how to clear and reload drones

Clear all drones.

In [ ]:
simulator.clear_drones()

Load drones again from `students` directory.

In [ ]:
simulator.load_drones()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to rearrange the rings

Rearrange the rings.

In [ ]:
simulator.move_rings()

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

Run simulation until `max_time` is reached or until all drones finish.

In [ ]:
simulator.run(max_time=5.0)

## Example of how to take a snapshot

View from behind a particular drone.

In [ ]:
simulator.camera_droneview('template')

Display a snapshot.

In [ ]:
# Get snapshot as 480 x 480 x 4 numpy array of RGBA values
rgba = simulator.snapshot()

# Display snapshot
plt.figure(figsize=(9, 9))
plt.imshow(rgba)

Save the snapshot.

In [ ]:
plt.imsave('my_snapshot.png', rgba)

## Example of how to record a movie

You must install both [imageio](https://github.com/imageio/imageio) and [imageio-ffmpeg](https://github.com/imageio/imageio-ffmpeg) in order for this to work. You can do this from a terminal (in your `ae353-bullet` environment, for example) as follows:

```
pip install imageio
pip install imageio-ffmpeg
```

Reset drones (places drones at random positions within the start ring).

In [ ]:
simulator.reset()

View from behind a particular drone.

In [ ]:
simulator.camera_droneview('template')

Run simulation until `max_time` is reached or until all drones finish, saving movie to `my_video.mp4`.

In [ ]:
simulator.run(max_time=1.0, video_filename='my_video.mp4')